In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
 data = pd.read_table('/kaggle/input/titanic/train.csv', sep=',')

In [3]:
#Drop data that's irrelevant like name, id number, and ticket id
data.drop(columns = ['Name', 'PassengerId', 'Ticket','Cabin'], inplace=True)
data = data.dropna()
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
889,1,1,male,26.0,0,0,30.0000,C


In [4]:
#First is Gaussian Naive Bayes classifier
#Need to learn P(X|Y) = P(X1 | Y) * P(X2|Y) * ... * P(Xn|Y)
#Use MLE to estimate a parameter for every feature, feature value, and outcome
#Assuming that each feature value that is continuous is Gaussian with params (E[Xi|Y=Yk], E[(Xi-uik)^2 | Y=yk])
SMOOTH = 1
survived_df = data[data.Survived == 1]
died_df = data[data.Survived == 0]

In [5]:
#First we want to calculate parameters for discrete cases

survived_df_discrete = survived_df.drop(columns = ['Fare', 'Age'])
died_df_discrete = died_df.drop(columns = ['Fare', 'Age'])

survived_df_continuous = survived_df.loc[:, ['Survived', 'Fare', 'Age']]
died_df_continuous = died_df.loc[:, ['Survived', 'Fare', 'Age']]


In [6]:
for test in survived_df_discrete['Sex'].value_counts():
    print(test)

195
93


In [7]:
test_list = [Counter()] * 3
test_list[0][0] += 1
test_list[1]

Counter({0: 1})

In [8]:
#This dict maps discrete attribute names to their parameters
survive_discrete_list = []
for i in range(survived_df_discrete.shape[1]):
    survive_discrete_list.append(Counter())

num_survived = survived_df_discrete.shape[0]  
for row, series in survived_df_discrete.iterrows():
    for i, val in enumerate(series):
     #   print("i: {}".format(i))
      #  print("val: {}".format(val))
        (survive_discrete_list[i])[val] += 1


for counter in survive_discrete_list:
    smoothing_denom = len(counter) * SMOOTH
    for key in counter:
        counter[key] = (counter[key] + SMOOTH) / (num_survived + smoothing_denom)

#Calculate the prior for people that survived, ie num_survived/total_people
survive_prob = num_survived/(num_survived + died_df.shape[0])
died_prob = 1-survive_prob
print(survive_discrete_list)

[Counter({1: 1.0}), Counter({1: 0.41580756013745707, 3: 0.29553264604810997, 2: 0.28865979381443296}), Counter({'female': 0.6758620689655173, 'male': 0.32413793103448274}), Counter({0: 0.5938566552901023, 1: 0.33447098976109213, 2: 0.040955631399317405, 3: 0.017064846416382253, 4: 0.013651877133105802}), Counter({0: 0.6313993174061433, 1: 0.21160409556313994, 2: 0.13651877133105803, 3: 0.013651877133105802, 5: 0.006825938566552901}), Counter({'S': 0.6941580756013745, 'C': 0.27491408934707906, 'Q': 0.030927835051546393})]


In [9]:
#Do the same for people that died
died_discrete_list = []

#number of people that died
num_died = died_df_discrete.shape[0]
for i in range(died_df_discrete.shape[1]):
    died_discrete_list.append(Counter())

for row, series in died_df_discrete.iterrows():
    for i, val in enumerate(series):
        died_discrete_list[i][val] += 1

for counter in died_discrete_list:
    smoothing_denom = len(counter) * SMOOTH
    for key in counter:
        counter[key] = (counter[key] + SMOOTH) / (num_died + smoothing_denom)

print(died_df_discrete)

     Survived  Pclass     Sex  SibSp  Parch Embarked
0           0       3    male      1      0        S
4           0       3    male      0      0        S
6           0       1    male      0      0        S
7           0       3    male      3      1        S
12          0       3    male      0      0        S
..        ...     ...     ...    ...    ...      ...
883         0       2    male      0      0        S
884         0       3    male      0      0        S
885         0       3  female      0      5        Q
886         0       2    male      0      0        S
890         0       3    male      0      0        Q

[424 rows x 6 columns]


In [10]:
#test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
#for i in test_data_discrete.itertuples(name="Passenger"):
#    print(i)
survive_discrete_list

[Counter({1: 1.0}),
 Counter({1: 0.41580756013745707,
          3: 0.29553264604810997,
          2: 0.28865979381443296}),
 Counter({'female': 0.6758620689655173, 'male': 0.32413793103448274}),
 Counter({1: 0.33447098976109213,
          0: 0.5938566552901023,
          4: 0.013651877133105802,
          3: 0.017064846416382253,
          2: 0.040955631399317405}),
 Counter({0: 0.6313993174061433,
          2: 0.13651877133105803,
          1: 0.21160409556313994,
          5: 0.006825938566552901,
          3: 0.013651877133105802}),
 Counter({'C': 0.27491408934707906,
          'S': 0.6941580756013745,
          'Q': 0.030927835051546393})]

In [36]:
#Can evaluate for discrete variables
#Y = arg max_yk( P(Y=y_k) * P(X_i|Y=y_k))
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
features_list = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
test_data_discrete = test_data.loc[:, features_list]
def test_discrete_GNB(new_example):
    cond_prob_survive = 1
    cond_prob_died = 1
    for i, attribute in enumerate(new_example):
        
        if i == 0:
            continue
        cond_prob_survive *= survive_discrete_list[i][attribute]
        cond_prob_died *= died_discrete_list[i][attribute]
    curr_survive_prob = survive_prob * cond_prob_survive
    curr_died_prob = died_prob * cond_prob_died
    return (new_example.PassengerId, 1 if curr_survive_prob > curr_died_prob else 0)


In [27]:
test_data_discrete.loc[0,:]

Pclass         3
Sex         male
SibSp          0
Parch          0
Embarked       Q
Name: 0, dtype: object

In [65]:
results_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
test_data_discrete2 = test_data.loc[:, ['PassengerId'] + features_list]
for _, test_example in test_data_discrete2.iterrows():
    temp = test_discrete_GNB(test_example)
    new_row = pd.DataFrame({'PassengerId': [temp[0]], 'Survived': [temp[1]]})
    results_df = results_df.append(new_row)
 #   results_dict[temp[0]] = temp[1]
results_df.to_csv('./discrete_results.csv', index = False)
#results_df.drop()
#results_df.to_csv()

In [47]:
result_df = pd.DataFrame.from_dict(data = results_dict,orient='index')
result_df
#result_df.columns = ['PassengerId', 'Survived']  

,0
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0
1308,0
